<a href="https://colab.research.google.com/github/fcoliveira-utfpr/climate_parana/blob/main/area_of_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start – Libraries
---

In [1]:
# -------------------------
# IMPORTS
# -------------------------
!pip install geobr geopandas rasterio elevation cartopy --quiet
import geobr
import geopandas as gpd
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.0/338.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.1 MB/s eta 0:00:00


# Data source
---

In [8]:
# ----------------------------
# Data Loading (geobr)
# ----------------------------
# Municipalities of Paraná
mun_pr = geobr.read_municipality(code_muni="PR", year=2020)

# Biomes
biomas = geobr.read_biomes(year=2019)

# Conservation Units
ucs = geobr.read_conservation_units()

# Indigenous Lands
ti = geobr.read_indigenous_land()

# ----------------------------
# Appropriate projection for area calculation
# ----------------------------
epsg_proj = 31982  # SIRGAS 2000 / UTM zone 22S (Paraná)

mun_pr = mun_pr.to_crs(epsg_proj)
biomas = biomas.to_crs(epsg_proj)
ucs = ucs.to_crs(epsg_proj)
ti = ti.to_crs(epsg_proj)

# ----------------------------
# Municipality area (km²)
# ----------------------------
mun_pr["area_km2"] = mun_pr.geometry.area / 1e6

# ----------------------------
# Dominant biome per municipality
# ----------------------------
inter_bioma = gpd.overlay(mun_pr, biomas, how="intersection")
inter_bioma["area_intersec"] = inter_bioma.geometry.area

bioma_dom = (
    inter_bioma
    .sort_values("area_intersec", ascending=False)
    .drop_duplicates("code_muni")[["code_muni", "name_biome"]]
)

mun_pr = mun_pr.merge(bioma_dom, on="code_muni", how="left")

# ----------------------------
# Presence of Conservation Units and Indigenous Lands
# ----------------------------
mun_pr["has_conservation_unit"] = mun_pr.intersects(ucs.union_all())
mun_pr["has_indigenous_land"] = mun_pr.intersects(ti.union_all())

# ----------------------------
# Final result as DataFrame
# ----------------------------
df_parana_environment_rural = mun_pr.drop(columns="geometry")

# Preview
df_parana_environment_rural.head()

,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,area_km2,name_biome,has_conservation_unit,has_indigenous_land
0,4100103.0,Abatiá,41.0,PR,Paraná,4.0,Sul,228.518093,Mata Atlântica,False,True
1,4100202.0,Adrianópolis,41.0,PR,Paraná,4.0,Sul,1350.483016,Mata Atlântica,True,False
2,4100301.0,Agudos Do Sul,41.0,PR,Paraná,4.0,Sul,192.357285,Mata Atlântica,False,False
3,4100400.0,Almirante Tamandaré,41.0,PR,Paraná,4.0,Sul,194.757763,Mata Atlântica,True,False
4,4100459.0,Altamira Do Paraná,41.0,PR,Paraná,4.0,Sul,387.010301,Mata Atlântica,False,False


# Summarized data
---

In [10]:
# ============================================================
# SUMMARY DATAFRAME – ENVIRONMENT & TERRITORY (PARANÁ)
# ============================================================

# Number of municipalities
n_mun = mun_pr.shape[0]

# Total state area (km²)
area_total_estado_km2 = mun_pr["area_km2"].sum()

# Average municipality area
area_media_municipios_km2 = mun_pr["area_km2"].mean()

# Municipalities by dominant biome
biomas_mun = (
    mun_pr["name_biome"]
    .value_counts()
    .to_dict()
)

# Percentage of municipalities with Conservation Units
perc_mun_uc = mun_pr["has_conservation_unit"].mean() * 100

# Percentage of municipalities with Indigenous Lands
perc_mun_ti = mun_pr["has_indigenous_land"].mean() * 100

# ----------------------------
# Final DataFrame
# ----------------------------
df_resumo_parana = pd.DataFrame({
    "state": ["Paraná"],
    "number_of_municipalities": [n_mun],
    "total_area_km2": [area_total_estado_km2],
    "average_municipality_area_km2": [area_media_municipios_km2],
    "percentage_municipalities_with_conservation_units": [perc_mun_uc],
    "percentage_municipalities_with_indigenous_lands": [perc_mun_ti],
    "municipalities_by_biome": [biomas_mun]
})

df_resumo_parana

,state,number_of_municipalities,total_area_km2,average_municipality_area_km2,percentage_municipalities_with_conservation_units,percentage_municipalities_with_indigenous_lands,municipalities_by_biome
0,Paraná,399,199273.987969,499.433554,24.81203,9.273183,"{'Mata Atlântica': 398, 'Cerrado': 1}"
